# MLP Model


In [1]:
import torch

In [1]:
def softmax(z):
    z_exp = torch.exp(z)
    z_exp_sum = torch.sum(z_exp, dim=1) + 1e-8  # softmax 분모
    a = z_exp / z_exp_sum.unsqueeze(dim=1)
    return a

In [2]:
def cross_entropy(y_hat, y):
    loss = torch.sum(-y * torch.log(y_hat), dim=1)
    loss = loss.mean()
    return loss

# MLP Model

## Neural Network Design

- Pytorch에서 neural network는 torch.nn.Module이라는 베이스 클래스를 상속받아서 사용하고 각 요소들은 torch.nn에서 사용할 수 있음


In [1]:
import torch.nn as nn
import torch.nn.functional as F


class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.linear1 = nn.Linear(10, 20)
        self.linear2 = nn.Linear(20, 1)

    def forward(self, x):
        x = F.sigmoid(self.linear1(x))
        return F.sigmoid(self.linear2(x))

- Linear Layer
  - Fully connected layer라고 불름
  - torch.nn.Linear(in_features, out_features, bias=True)
  - in_features: 입력 perceptron의 수
  - out_features: 출력 perceptron의 수
  - bias: bias를 사용할 것인지 여부


- Activation function
  - Sigmoid
    - torch.nn.function.sigmoid
    - 0~1
  - Tanh
    - torch.nn.function.tanh
    - -1~1
  - ReLU
    - torch.nn.function.relu
    - max(0, x)


In [2]:
class MLPnet(nn.Module):  # nn.Module을 상속받음
    def __init__(
        self,
    ):  # 생성자 정의 (해당 클래스의 인스턴스가 생성될 때 자동으로 호출)
        super(MLPnet, self).__init__()  # nn.Module의 생성자(__init__)를 호출
        self.fc1 = nn.Linear(
            in_features=3 * 32 * 32, out_features=128
        )  # 3*32*32 >> 128
        self.fc2 = nn.Linear(in_features=128, out_features=64)  # 128 >>> 64
        self.dropout = nn.Dropout(p=0.5)
        self.fc3 = nn.Linear(in_features=64, out_features=10)  # 64 >>> 10

    def forward(self, x):  # forward propagation
        x = torch.flatten(x, 1)  # 행렬형태의 입력을 벡터형태로 변형
        x = self.fc1(x)  # 1st layer
        x = F.relu(x)
        x = self.fc2(x)  # 2nd layer
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)  # 3rd layer
        return x

- Forward Path
  - 입력부터 출력까지의 순방향의 과정을 의미
  - Forward path를 통해서 최종 Loss를 계산할 수 있음


- Backward Path
  - Forward path로부터 게산된 Loss에 대한 각 파라미터 W, b의 gradient 계산해가는 과정(Backpropagation 이라고도 불림)
  - 처음부터 계산하는것은 너무 어려움
  - 이때 Chain Rule을 이용하면 다음 계산값은 이전 계산값을 활용해서 계산할 수 있음
  - Update는 기존 W, b값에서 계산한 gradient의 반대 방향으로 학습률(lerning rate)를 곱하여 변화


- torch.optim
  - 모델을 업데이트하는 다양한 optimization algorithm을 포함


In [3]:
import torch.optim as optim

- SGD(Stochastic Gradient Descent)
  - 확률적 경사 하강법 알고리즘
  - 전체 데이터를 전부 고려하지 않고 미니배치에 대한 gradient 기반으로 업데이트 함
  - 미니 배치를 선택하는 것이 확률적이기 때문에 Stochastic이라는 표현이 붙음
  - Pytorch에서는 Full-batch Gradient Descent(FGD)를 따로 제공하지 않고 사용자가 미니배치 사이즈를 전체 데이터로 지정하면 FGD로 사용할 수 있음
  - Optimizer에는 Model의 parameters의 정보를 입력으로 넣어주어야 함
  - 학습률(learning rate)을 지정하고, 그 외에 momentum, weight_decay, Nesterov등의 옵션을 고려


In [ ]:
model = MLPnet()
optimizer = optim.SGD(
    model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0005, nesterov=True
)

- weight decay
  - 모든 레이어의 W 파라미터에 대해서 적용되며 W 값이 과도하게 커지지 않도록 W에 L2-norm을 적용시킨 크기 값을 Loss식에 추가한 것


# Adam(Adaptive Moment Estimation)

- momentum의 방향과 크기를 모두 고려한 Optimization 알고리즘
- SGD와 마찬가지고 lr을 설정할 수 있고, betas라는 parameters가 있으며 기본 값인(0.9, 0.999)를 사용하는 것을 권장
- Gradient에 따라서 적응적으로 업데이트 값을 적용하는 효과가 있어서 이론적으로는 learning rate값에 민감하지 않고 큰 값을 적용하는 것도 가능
- Adam이 SGD보다 더 나중에 고안된 optimizer이지만 항상 Adam이 더 좋은 성능을 내는 것이 아님(Dataset이나 Task에 따라 다름)


In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999))

# Scheduler

- 학습의 진행정도에 따라 learning rate를 조절하는 역할
- StepLR:
  - torch.optim.lr_scheduler.StepLR(optimizer, step_size=40, gamma=0.1)
  - step_size에 도달하면 learning rate에 gamma를 곱함
- MultiStepLR
  - torch.optim.lr_schduler.MultiStepLR(optimizer, milestones=[30,80], gamma=0.1)
  - 복수의 step_size를 milestones으로 정의하여 각 milestones에 도달하면 learning rate에 gamma를 곱함
- CosineAnnealingLR:
  - torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)
  - Cosine 개형을 따라가며 learning rate가 정해짐
  - T_max: cosine 주기의 1/2


- StepLR의 경우 Step size를 어떻게 정하는가?
  - Loss가 줄어들지 않는 포인트에서 일반적으로 Learning rate를 낮춰줌
  - 주의점은 Loss가 작은 것이 항상 좋은 것은 아님
  - Trainig Loss는 작지만 Test에서 성능이 좋지 못할 수도 있음(overfitting)


In [4]:
import torch
from torchvision import datasets


# 데이터 불러오기: torchvision 패키지에서 cifar10의 train data 와 test data를 불러옴
def load_data(dataset, dataset_path):
    train_dataset = None
    test_dataset = None

    if dataset == "CIFAR10":
        train_dataset = datasets.CIFAR10(
            dataset_path,  # 해당 데이터 셋의 path
            download=True,  # 해당 데이터셋을 다운 받을 것인지
            train=True,
        )  # 학습 용도로 사용될 것인지
        train_dataset.data = torch.tensor(
            train_dataset.data
        )  # numpy array 형태를 torch tensor로 변경
        train_dataset.data = torch.permute(
            train_dataset.data, dims=(0, 3, 1, 2)
        )  # (B,H,W,C) -> (B,C,H,W) 형태로 변경
        train_dataset.targets = torch.tensor(train_dataset.targets)

        test_dataset = datasets.CIFAR10(dataset_path, download=False, train=False)
        test_dataset.data = torch.tensor(test_dataset.data)
        test_dataset.data = torch.permute(test_dataset.data, dims=(0, 3, 1, 2))
        test_dataset.targets = torch.tensor(test_dataset.targets)

    else:
        print("Incorrect dataset!")
    return train_dataset, test_dataset

In [ ]:
# Input Preprocess: Model이 학습할 수 있도록 input으로 들어가는 train data를 수정함
def input_preprocess(x, train_mode=True):
    x = x / 255.0  # MinmaxScaler : input data 를 0~1사이의 float로 바꿔주기
    if train_mode:
        half = int(x.shape[0] / 2)  # 미니배치 개수의 반
        x[0:half, :] = torch.flip(x[0:half, :], dims=[3])  # (B,C,H,W)의 W를 반전

    return x

In [ ]:
def onehot_encoding(y, n_class):
    out = torch.zeros(
        len(y), n_class
    )  # 전체 클래스 수 만큼 원소를 갖는 zero vector 들을 생성
    for i in range(len(y)):  # 미니배치 수(y의 길이) 만큼 반복
        out[i, y[i]] = 1  # target index를 1로 채워두기
    y = out.float()  # 추후 연산을 위해서 dtype을 float으로 변환
    return y

In [ ]:
def train(model, optimizer, scheduler, train_iter, train_samp, train_dataset):
    model.train(True)  # training mode on (True : train, False : test)
    class_number = len(train_dataset.classes)

    # Data를 load하는 부분에서 input data와 label 각각에 .to(device)를 적용
    # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    for i in range(train_iter):
        batch_index = train_samp.get_random_idx()  # mini-batch만큼 index 가져오기
        x, label = train_dataset.data[batch_index], train_dataset.targets[batch_index]

        # x = x.to(device)
        # label = label.to(device)

        x = input_preprocess(x)  # data preprocessing
        label = onehot_encoding(label, class_number)  # one-hot encoding
        logits = model(x)  # forward propagation (model.forward(x)와 동일)
        inference = softmax(logits)  # 전체의 합이 1이 되도록 확률값으로 변환(softmax)
        loss = cross_entropy(inference, label)  # Loss 계산
        optimizer.zero_grad()  # gradient 0으로 초기화. 초기화 하지 않으면 쌓일수 있음
        loss.backward()  # back propagation (그래디언트 계산 -> 어느 방향으로 업데이트 하면 될지 계산됨)
        optimizer.step()  # update parameters(여기서 업데이트를 함)
        scheduler.step()  # scheduler의 step+1

    return model

- model.train(True)로 모델의 학습/비학습 상태임을 정의
- 클래스의 수를 변수로 지정


## Test함수 구현

- Test는 train과 달리 랜덤하게 data를 추출할 필요 없음
- 미니배치 size로 나누어 남는 data에 대해서도 반드시 forward path를 진행(Computing power가 충분하면 full-batch로 한번에 계산해도 무방)
- softmax를 통과하고 나온 예측 값 중 확률이 가장 높은 index를 추측 값으로 예측
  - ex)[0.1,0.6,0.09, 0.04, 0.05, 0.01, 0.04, 0.05, 0.01, 0.01] -> 두번째 class로 예측
- Accuracy = (맞춘 개수) / (전체 개수)


In [ ]:
def test(model, test_dataset, b_size):
    model.train(False)  # training mode off
    test_acc = 0.0
    test_iterations = len(test_dataset) // b_size  # 총 iterations 수 계산
    for i in range(test_iterations + 1):  # 나머지 한 번에 대해서 추가
        test_x = test_dataset.data[i * b_size : i * b_size + b_size]
        test_label = test_dataset.targets[i * b_size : i * b_size + b_size]
        test_x = input_preprocess(test_x, False)
        test_output = model(test_x)  # forward path
        _, preds = torch.max(test_output, 1)  # softmax를 생략하고 최대값의 index 찾음
        test_acc += torch.sum(
            preds == test_label.data
        )  # mini-batch 안에서 맞춘 개수를 더해줌

    print(
        f"Test Accuracy: {test_acc / len(test_dataset) * 100:.2f}%"
    )  # 최종 성능을 백분율로 출력

- model.train(False) 로 모델의 비학습 상태임을 정의
- test_acc를 0으로 설정하여 맞춘 개수를 카운트
- test 데이터를 미니배치로 나눈 나머지에 대해서도 추론 값을 평가해야 하므로 test_iterations에 1을 더한 값으로 for loop 수행


In [ ]:
import RandomSampler

model = MLPnet()
# Model을 to(device)를 통해서 cuda가 available한 경우 적용
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.5)

dataset_path = "./cifar10_data/"
dataset_name = "CIFAR10"
train_dataset, test_dataset = load_data(dataset_name, dataset_path)

train_batch_size = 64
test_batch_size = 64

# 배치사이즈가 크면 클수록 epoch안에 도는 iteration 수가 줄어듬
# 배치사이즈가 작게 되면 많은 iteration을 돌려야지 비로소 한 번에 epoch을 돌 수 있음
train_iter = 3000
train_samp = RandomSampler(len(train_dataset), train_batch_size)

model = train(model, optimizer, scheduler, train_iter, train_samp, train_dataset)
test(model, test_dataset, test_batch_size)